# Neural Network model for fluid dynamics (Part 2)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/paolodeangelis/Sistemi_a_combustione/blob/main/2.2-NN_Reynolds_P2.ipynb)

<img src="https://github.com/paolodeangelis/Sistemi_a_combustione/raw/main/assets/img/warning-work-in-progress.jpg" width="600"/>